In [143]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [144]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [145]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [146]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [147]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_market_data)

In [148]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(df_scaled, columns=df_market_data.columns)

# Copy the crypto names from the original data
df_scaled['coin_name'] = df_market_data.index

# Display sample data
df_scaled.head()



,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,coin_name
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,bitcoin
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,ethereum
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,tether
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,ripple
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,bitcoin-cash


---

### Find the Best Value for k Using the Original Data.

In [149]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [150]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=0)
    
    # Fit the model to the scaled data
    model.fit(df_scaled.drop(columns=['coin_name']))  # Drop the 'coin_name' column
    
    # Append the model.inertia_ to the inertia list
    inertia_values.append(model.inertia_)

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  s

In [151]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Display the DataFrame
df_elbow


,k,inertia
0,1,287.000000
1,2,198.571818
2,3,123.190482
3,4,79.022435
4,5,65.302379
5,6,52.101806
6,7,46.035250
7,8,37.284807
8,9,32.161417
9,10,28.075253


In [152]:
# Plot the Elbow curve using hvplot
elbow_plot = df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=df_elbow['k'].tolist(),
    grid=True
)

elbow_plot.opts(width=800, height=400)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3 or 4 would be the best k value

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [153]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters= 3, random_state=0)

In [154]:
# Fit the K-Means model using the scaled data
df_numeric = df_scaled.drop(columns=['coin_name'])
kmeans_model.fit(df_numeric)

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=0)

In [155]:
# Predict the clusters using the scaled data
clusters = kmeans_model.predict(df_numeric)

# Print the resulting array of cluster values
print(clusters)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0
 1 1 1 1]


In [156]:
# Create a copy of the DataFrame
df_clustered = df_scaled.copy()

In [157]:
# Add a new column to the DataFrame with the predicted clusters
df_clustered['predicted_cluster'] = clusters

# Display sample data
df_clustered.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,coin_name,predicted_cluster
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,bitcoin,1
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,ethereum,1
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,tether,1
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,ripple,1
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,bitcoin-cash,1


In [158]:
# Create a scatter plot using hvPlot
scatter_plot = df_clustered.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    color="predicted_cluster",
    hover_cols=["coin_name"],
    title="Cryptocurrency Clusters"
)

# Display the scatter plot
scatter_plot


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_cluster,coin_name)

---

### Optimize Clusters with Principal Component Analysis.

In [159]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [160]:
# Use the PCA model with fit_transform to reduce to three principal components
pca_features = pca_model.fit_transform(df_numeric)

# Convert the PCA features array to a DataFrame
df_pca = pd.DataFrame(
    data=pca_features,
    columns=['PCA1', 'PCA2', 'PCA3'],
    index=df_scaled.index
)

# View the first five rows of the DataFrame
df_pca.head()

,PCA1,PCA2,PCA3
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


In [161]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca_model.explained_variance_ratio_
print(explained_variance)


[0.3719856  0.34700813 0.17603793]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Very little variance is explained with the three principal components

In [162]:
# Creating a DataFrame with the PCA data
df_pca = pd.DataFrame(pca_features, columns=['PCA1', 'PCA2', 'PCA3'])

# Copy the crypto names from the original data
df_pca['coin_name'] = df_market_data.index

# Set the coin_name column as index
df_pca.set_index('coin_name', inplace=True)

# Display sample data
df_pca.head()

,PCA1,PCA2,PCA3
coin_name,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [163]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))


In [164]:
inertia_values = []

for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(df_pca)
    inertia_values.append(model.inertia_)

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  s

In [165]:
# Create a dictionary to store the data for the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inertia': inertia_values
}

# Create a DataFrame using the elbow_data dictionary
new_df_elbow = pd.DataFrame(elbow_data)

In [166]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
new_df_elbow.hvplot.line(
    x='k_values',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=k_values
)

:Curve   [k_values]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 3


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [167]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=3, random_state=0)

In [168]:
# Fit the K-Means model using the PCA data
kmeans_model.fit(df_pca)

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=0)

In [169]:
# Predict the clusters using the PCA data
predicted_clusters = kmeans_model.predict(df_pca)

# Print the resulting array of cluster values
print(predicted_clusters)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0
 1 1 1 1]


In [170]:
# Create a copy of the DataFrame with the PCA data
df_pca_clustered = df_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_clustered['predicted_cluster'] = predicted_clusters

# Display sample data
print(df_pca_clustered.head())


                  PCA1      PCA2      PCA3  predicted_cluster
coin_name                                                    
bitcoin      -0.600667  0.842760  0.461595                  1
ethereum     -0.458261  0.458466  0.952877                  1
tether       -0.433070 -0.168126 -0.641752                  1
ripple       -0.471835 -0.222660 -0.479053                  1
bitcoin-cash -1.157800  2.041209  1.859715                  1


In [171]:
# Create a scatter plot using hvPlot
scatter_plot = df_pca_clustered.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    color='predicted_cluster',
    hover_cols=['coin_name'],
    title='PCA Scatter Plot with Predicted Clusters'
)

# Show the scatter plot
scatter_plot


:Scatter   [PCA1]   (PCA2,predicted_cluster,coin_name)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [172]:
# Composite plot to contrast the Elbow curves
# Create a DataFrame for the Elbow curves
elbow_plot = df_elbow.hvplot.line(
    x='k',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=df_elbow['k'].tolist(),
    grid=True
)

new_elbow_plot = new_df_elbow.hvplot.line(
    x='k_values',
    y='inertia',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    xticks=k_values,
    grid=True
)

elbow_plot.opts(width=800, height=400)
new_elbow_plot.opts(width=800, height=400)
composite_plot = (new_elbow_plot + elbow_plot).cols(1)

composite_plot

:Layout
   .Curve.I  :Curve   [k_values]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [173]:
# Composite plot to contrast the clusters
scatter_plot = df_clustered.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    color="predicted_cluster",
    hover_cols=["coin_name"],
    title="Cryptocurrency Clusters"
)

new_scatter_plot = df_pca_clustered.hvplot.scatter(
    x='PCA1',
    y='PCA2',
    color='predicted_cluster',
    hover_cols=['coin_name'],
    title='PCA Scatter Plot with Predicted Clusters'
)

scatter_plot.opts(width=800, height=400)
new_scatter_plot.opts(width=800, height=400)
composite_plot = (new_scatter_plot + scatter_plot).cols(1)

composite_plot

:Layout
   .Scatter.I  :Scatter   [PCA1]   (PCA2,predicted_cluster,coin_name)
   .Scatter.II :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_cluster,coin_name)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using less clusters led to a denser cluster with fewer outliers.